# MACD
This notebook use MACD strategy to backtest the performance

In [9]:
# Run it when crypto-backtest is updated, and then restart the kernel
!pip uninstall crypto-backtest -y
!pip install git+https://github.com/BullionBear/crypto-backtest.git

Found existing installation: crypto-backtest 0.0.16
Uninstalling crypto-backtest-0.0.16:
  Successfully uninstalled crypto-backtest-0.0.16
  Cloning https://github.com/BullionBear/crypto-backtest.git to /tmp/pip-req-build-5h7wwlhv
  Running command git clone --filter=blob:none --quiet https://github.com/BullionBear/crypto-backtest.git /tmp/pip-req-build-5h7wwlhv
  Resolved https://github.com/BullionBear/crypto-backtest.git to commit d3bf780d7a2acdac086d77eb8facc9cc3fb204df
  Preparing metadata (setup.py) ... done
  Created wheel for crypto-backtest: filename=crypto_backtest-0.0.17-py3-none-any.whl size=10169 sha256=fbc2d201298378c081dc72bfff5ce44f56148e3f039da5b9a0bdc117e2c5a6a8
  Stored in directory: /tmp/pip-ephem-wheel-cache-rbz7vo1m/wheels/14/d1/5f/193347d44b51670dd287000ac535cdb7a8fc3f84e426532382
Successfully built crypto-backtest

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import backtest
backtest.__version__

'develop_0.0.17+'

In [2]:
from backtest.components.core import ZipKLineIterator
from backtest.components import KLineMarket

from datetime import datetime, timezone

start_time = datetime(2023, 7, 1, tzinfo=timezone.utc).timestamp() * 1000
end_time = datetime(2023, 9, 1, tzinfo=timezone.utc).timestamp() * 1000
data_source = '/home/yite/crypto_data/binance/data'
kline_iterator = ZipKLineIterator('BTCUSDT', start_time, end_time, data_source)
market = KLineMarket(kline_iterator)

In [3]:
from backtest.components.core import SMAFiltration

fast_div = SMAFiltration(5)
slow_div = SMAFiltration(10)
for _ in range(10 + 1):
    if market.is_kline():
        kline = market.get_kline()
        fast_div.put(kline)
        slow_div.put(kline)
print(f"fast_div = {fast_div.get()}, slow_div = {slow_div.get()}")

TypeError: Argument 'real' has incorrect type (expected numpy.ndarray, got list)